## Create CNN Model and Optimize it using Keras Tuner

In [1]:
import tensorflow as tf
from tensorflow import keras
import numpy as np
import pandas as pd

In [2]:
print(tf.__version__)

2.3.0


In [3]:
fashion_mnist=keras.datasets.fashion_mnist

In [4]:
(train_images,train_labels),(test_images,test_labels)=fashion_mnist.load_data()

4423680/4422102 [==============================] - 0s 0us/step:  - ETA


In [5]:
train_images=train_images/255.0
test_images=test_images/255.0

In [6]:
train_images[0].shape

(28, 28)

In [7]:
train_images=train_images.reshape(len(train_images),28,28,1)
test_images=test_images.reshape(len(test_images),28,28,1)

In [8]:
def build_model(hp):  
  model = keras.Sequential([
    keras.layers.Conv2D(
        filters=hp.Int('conv_1_filter', min_value=32, max_value=128, step=16),
        kernel_size=hp.Choice('conv_1_kernel', values = [3,5]),
        activation='relu',
        input_shape=(28,28,1)
    ),
    keras.layers.Conv2D(
        filters=hp.Int('conv_2_filter', min_value=32, max_value=64, step=16),
        kernel_size=hp.Choice('conv_2_kernel', values = [3,5]),
        activation='relu'
    ),
    keras.layers.Flatten(),
    keras.layers.Dense(
        units=hp.Int('dense_1_units', min_value=32, max_value=128, step=16),
        activation='relu'
    ),
    keras.layers.Dense(10, activation='softmax')
  ])
  
  model.compile(optimizer=keras.optimizers.Adam(hp.Choice('learning_rate', values=[1e-2, 1e-3])),
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])
  
  return model

In [10]:
from keras_tuner import RandomSearch
from keras_tuner.engine.hyperparameters import HyperParameters

In [11]:
tuner_search=RandomSearch(build_model,
                          objective='val_accuracy',
                          max_trials=5,directory='output',project_name="Mnist Fashion")

In [12]:
tuner_search.search(train_images,train_labels,epochs=3,validation_split=0.1)

Trial 5 Complete [00h 04m 01s]
val_accuracy: 0.9113333225250244

Best val_accuracy So Far: 0.9113333225250244
Total elapsed time: 00h 20m 51s
INFO:tensorflow:Oracle triggered exit


In [13]:
model=tuner_search.get_best_models(num_models=1)[0]

In [14]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 26, 26, 64)        640       
_________________________________________________________________
conv2d_1 (Conv2D)            (None, 24, 24, 48)        27696     
_________________________________________________________________
flatten (Flatten)            (None, 27648)             0         
_________________________________________________________________
dense (Dense)                (None, 112)               3096688   
_________________________________________________________________
dense_1 (Dense)              (None, 10)                1130      
Total params: 3,126,154
Trainable params: 3,126,154
Non-trainable params: 0
_________________________________________________________________


In [ ]:
model.fit(train_images, train_labels, epochs=10, validation_split=0.1, initial_epoch=3)

Train on 54000 samples, validate on 6000 samples
Epoch 4/10
54000/54000 [==============================] - 8s 148us/sample - loss: 0.1383 - accuracy: 0.9486 - val_loss: 0.2772 - val_accuracy: 0.9073
Epoch 5/10
54000/54000 [==============================] - 8s 142us/sample - loss: 0.0999 - accuracy: 0.9637 - val_loss: 0.2729 - val_accuracy: 0.9107
Epoch 6/10
54000/54000 [==============================] - 8s 144us/sample - loss: 0.0719 - accuracy: 0.9734 - val_loss: 0.3503 - val_accuracy: 0.9120
Epoch 7/10
54000/54000 [==============================] - 7s 138us/sample - loss: 0.0520 - accuracy: 0.9811 - val_loss: 0.3996 - val_accuracy: 0.9090
Epoch 8/10
54000/54000 [==============================] - 8s 142us/sample - loss: 0.0396 - accuracy: 0.9860 - val_loss: 0.4146 - val_accuracy: 0.9103
Epoch 9/10
54000/54000 [==============================] - 8s 143us/sample - loss: 0.0336 - accuracy: 0.9881 - val_loss: 0.4527 - val_accuracy: 0.9142
Epoch 10/10
54000/54000 [==========================